# Optvier - Trading at close

## Introduction

First we should thanks optiver for holding this competition for Data science/Quant community. The aim of this notebook is to *predict* the closing price of stocks. The reason of predicting the closing prices is that there is an evidence of large amount of trades are made in last ten minutes before the close of the market. Hence, closing prices can be served as an indicator for traders to evaluate the stocks. 

## Work Flow of this notebook 
There are several thing I would like to achieve in this notebook. The first is a simple elementart data analysis (EDA) of the stock data. There are some features which deserve an attention such as the `imbalance volume`, `ask/bid size`, and `axsk/bid price`, which are the essential features to make an `order book`. Secondly, based on the EDA, the next step is feature engineering, buyt I would say `feature reverse-engineering`. There are lots of notebook availables in the competitions and it's good to `reverse engineer` the mindset of building those features and then try to build by our own. Once features engineering is done, we come to the model selection and training. I have to ideas in my mind: `transformer AE kind RNN` or `XGBoost`. They are the paradigms of time-series forecasting but I prefer XGBoost due to its simplicity. Finally, we will use the test set to make prediction and submit to the organization. To sum up, our work flow is: 
1. Data Preprocessing and Elementary Data Analysis
2. Features Engineering 
3. Model Selection [`XGBoost`]
4. Prediction and Submission


In [ ]:
# Data Analytic Tools
import matplotlib.pyplot as plt 
import pandas as pd
import seaborn as sns 
import plotly.express as px
import plotly.graph_objects as go
%matplotlib inline
%config InlineBackend.figure_format='retina'

# Computing Tools 
import lightgbm as lgb 
import xgboost as xgb 
import catboost as cbt 
import numpy as np 
import joblib 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Section 1: Data Preprocessing and Elementary Data Analysis

## Dataset Introduction 

- `stock_id` - A unique identifier for the stock. Not all stock IDs exist in every time bucket.
- `date_id` - A unique identifier for the date. Date IDs are sequential & consistent across all stocks.
- `imbalance_size` - The amount unmatched at the current reference price (in USD).
- `imbalance_buy_sell_flag` - An indicator reflecting the direction of auction imbalance.
    * buy-side imbalance; 1
    * sell-side imbalance; -1
    * no imbalance; 0
- `reference_price` - The price at which paired shares are maximized, the imbalance is minimized and the distance from the bid-ask midpoint is minimized, in that order. Can also be thought of as being equal to the near price bounded between the best bid and ask price.
- `matched_size` - The amount that can be matched at the current reference price (in USD).
- `far_price` - The crossing price that will maximize the number of shares matched based on auction - - - interest only. This calculation excludes continuous market orders.
- `near_price` - The crossing price that will maximize the number of shares matched based auction and continuous market orders.
- `[bid/ask]_price` - Price of the most competitive buy/sell level in the non-auction book.
- `[bid/ask]_size` - The dollar notional amount on the most competitive buy/sell level in the non-auction book.
- `wap` - The weighted average price in the non-auction book.
$$
\frac{\text{Bid Price } \times \text{Ask Size}  +  \text{Ask Price } \times \text{Bid Size} }{ \text{Bid Size + Ask Size}}
$$
- `seconds_in_bucket` - The number of seconds elapsed since the beginning of the day's closing auction, always starting from 0.
- `target` - The 60 second future move in the wap of the stock, less the 60 second future move of the synthetic index. Only provided for the train set.
    * The synthetic index is a custom weighted index of Nasdaq-listed stocks constructed by Optiver for this competition.
    * The unit of the target is basis points, which is a common unit of measurement in financial markets. A 1 basis point price move is equivalent to a 0.01% price move.
    * Where t is the time at the current observation, we can define the target:
        $$
        \text{Target} =  \Big( \frac{\text{Stock WAP}_{t + 60}}{\text{Stock WAP}_{t}} - \frac{\text{Index WAP}_{t+60} }{ \text{Index WAP}_t} \Big) * 10000
        $$
        All size related columns are in USD terms.


References:
- [High Frequency Trading II: Limit Order Book](https://www.quantstart.com/articles/high-frequency-trading-ii-limit-order-book/)


In [ ]:
# load the train.csv file using pandas build in function 
train = pd.read_csv('/kaggle/input/optiver-trading-at-the-close/train.csv')

# Print the first five value row of the dataset 
train.head()

# Print out the nan/ null values counts in each features
train.isnull().sum()

# Drop the nan/null values in the target since it would help for the inference 
trian = train.dropna(subset=['target'])

# Drop the index inplace 
train.reset_index(drop=True, inplace=True)

# 1.1 Data Preprocessing with Numba/JIT

In this subsession, we will preprocess the data and try to optimize the memory usages. One trick is to convert the data type in order to reduce the memory. 

# 1.2 Elemetary Data Analysis

In this subsession, we are going to inverstigate the the relation between each prices and sizes. Instead of tracking all stocks at all time, we examine the `stock[id==0]` and `date_id <= 5` as the exmaple notebook of Optiver. The reason of choosing `5 days` is that previously I picked `date_id ==0` but I could not see any negative imbalance. 

### Major Findings????
- [Iceberg Order? (short review in Chinese)](https://www.zhihu.com/question/23667442): In some moments, either the bid/ask sizes are several orders higher.




Reference Notebooks:
- [Introduction and Explore Data Analysis](https://www.kaggle.com/code/chiangken/introduction-and-explore-data-analysis)
- [Optiver 2023 | EDA | PyTorch: LSTM-Attention Model](https://www.kaggle.com/code/aniketkolte04/optiver-2023-eda-pytorch-lstm-attention-model)
- [Optiver Trading at Close Intro.](https://www.kaggle.com/code/tomforbes/optiver-trading-at-the-close-introduction)

In [ ]:
# define the df of stock00: 

stock00 = train.query('stock_id ==0 & date_id <5')
stock00

In [ ]:
# PLot the Prices changes of stock 0 at date 0 
fig = go.Figure()

fig.add_trace(
    go.Scatter(x = stock00['time_id'], 
            y = stock00['ask_price'], 
            name = 'ask price',
            line = dict(color = 'red')))

fig.add_trace(
    go.Scatter(x = stock00['time_id'], 
            y = stock00['bid_price'], 
            name = 'bid price',
            line = dict(color = 'green')))

fig.add_trace(
    go.Scatter(x = stock00['time_id'], 
            y = stock00['wap'], 
            name = 'WAP',
            line = dict(color = 'orange')))

fig.update_layout(title = "Overview for Ask/Bid Price and WAP") 

In [ ]:
# Examine the volume changes 
# PLot the Prices changes of stock 0 at date 0 
fig = go.Figure()

fig.add_trace(
    go.Scatter(x = stock00['time_id'], 
            y = stock00['ask_size'],  
            name = 'ask size',
            line = dict(color = 'red')))

fig.add_trace(
    go.Scatter(x = stock00['time_id'], 
            y = stock00['bid_size'], 
            name = 'bid size',
            line = dict(color = 'green')))

fig.update_layout(title = "Overview for Normalized Ask/Bid Sizes Comparison for 5 days (WHy large ask Size)")

In [ ]:
# Examine the volume changes 
# PLot the Prices changes of stock 0 at date 0 
fig = go.Figure()

fig.add_trace(
    go.Scatter(x = stock00['time_id'], 
            y = (stock00['ask_size'] - stock00['ask_size'].min()) / (stock00['ask_size'].max() - stock00['ask_size'].min()), 
            name = 'ask size',
            line = dict(color = 'red')))

fig.add_trace(
    go.Scatter(x = stock00['time_id'], 
            y = (stock00['bid_size'] - stock00['bid_size'].min()) / (stock00['bid_size'].max() - stock00['bid_size'].min()), 
            name = 'bid size',
            line = dict(color = 'green')))


fig.add_trace(
    go.Scatter(x = stock00['time_id'], 
            y = (stock00['wap'] - stock00['wap'].min()) / (stock00['wap'].max() - stock00['wap'].min()), 
            name = 'WAP',
            line = dict(color = 'orange')))



fig.update_layout(title = "Overview for Normalized Ask/Bid Sizes and WAP for 5 days")



In [ ]:
# Examine the volume changes 
# PLot the Prices changes of stock 0 at date 0 

total_size = stock00.eval('ask_size + bid_size').values

fig = go.Figure()

fig.add_trace(
    go.Scatter(x = stock00['time_id'], 
            y = (total_size - total_size.min()) / (total_size.max() - total_size.min()), 
            name = 'total size',
            line = dict(color = 'blue')))



fig.add_trace(
    go.Scatter(x = stock00['time_id'], 
            y = (stock00['wap'] - stock00['wap'].min()) / (stock00['wap'].max() - stock00['wap'].min()), 
            name = 'WAP',
            line = dict(color = 'orange')))

fig.update_layout(title = "Overview for Normalized Total Sizes and WAP for 5 days") 

In [ ]:
total_size = stock00.eval('ask_size + bid_size').values

fig = go.Figure()

fig.add_trace(
    go.Scatter(x = stock00['time_id'], 
            y = (stock00['target'] - stock00['target'].min()) \
               / (stock00['target'].max() - stock00['target'].min()) ,
            name = 'total size',
            line = dict(color = 'blue')))



fig.add_trace(
    go.Scatter(x = stock00['time_id'], 
            y = (stock00['wap'] - stock00['wap'].min()) / (stock00['wap'].max() - stock00['wap'].min()), 
            name = 'WAP',
            line = dict(color = 'orange')))

fig.update_layout(title = "Overview for Normalized Target and WAP for 5 days") 

# Section 2: Feature Engineering. 

Since there are no right answers of which features should be built. Therefore, based on some intuition I will make several features for example. Before going to the detail, lets see an equation which is the Geometric Brownian motion in differential form

$$
dS(t) = \mu S(t) dt + \sigma S(t) dW(t) \Leftrightarrow  S(t) = S_0 \exp \Big( (\mu - \frac{\sigma^2}{2}) t + \sigma W_t \Big ) 
$$

where $\mu, \sigma$, and $W(t)$ is the drift, volatility, and the standard Brownian motion. From CgatGPT, it gives us more concrete explanation on these variables:

- $\mu$ represents the drift component, which is the average or expected growth rate.
- $\sigma$ represents the volatility component, which measures the standard deviation of the variable's returns.
- $S(t)$ represents the value of the variable at time $t$

For the Brownian motion $W(t)$, we can think of it as an Binomial model where at each time $t$, we either winning or losing 1 dolalr. Therefore, we can write down the discrete form of Brownian motion $W(t)$: 

$$
W = X_1 + X_2 + \cdots + X_n ,~ X \in \{-1 ,1 \}  \sim  Bern(p) 
$$

By linearity of expectation and CLT, we can show that $W \sim \mathcal{N}(0, \text{some mean}) $, where $\mathbb{E} W = 0 $ for $p = 0.5$ ( equal chance of winning or losing)


# Section 2.X Create Featured Dataset 


Reference Notebooks:
- [🥇🥇baseline lgb, xgb and catboost🥇🥇](https://www.kaggle.com/code/yuanzhezhou/baseline-lgb-xgb-and-catboost)
- [Explained singel model⚡Optiver ](https://www.kaggle.com/code/zulqarnainali/explained-singel-model-optiver)

In [ ]:
from numba import njit, prange

# 📊 Function to compute triplet imbalance in parallel using Numba
def generate_features(df: pd.DataFrame) -> pd.DataFrame:
    features = ['seconds_in_bucket', 'imbalance_buy_sell_flag',
               'imbalance_size', 'matched_size', 'bid_size', 'ask_size',
                'reference_price','far_price', 'near_price', 'ask_price', 'bid_price', 'wap',
                'imb_s1', 'imb_s2'
               ]
    
    df['imb_s1'] = df.eval('(bid_size-ask_size)/(bid_size+ask_size)')
    df['imb_s2'] = df.eval('(imbalance_size-matched_size)/(matched_size+imbalance_size)')
    
    prices = ['reference_price','far_price', 'near_price', 'ask_price', 'bid_price', 'wap']
    
    # iceburg ask 
#     df['iceburg_ask'] = df['ask_size']
#     for i in range(len(df)):
#         if df['bid_size'][i] > 10 * df['ask_size'][i]: 
#             df['iceburg_ask'][i + 10] += df['bid_size'][i]
#     features.append('iceburg_ask')
            
#     # iceburg bid 
#     df['iceburg_bid'] = df['bid_size']
#     for i in range(len(df['ask_size'])):
#         if df['ask_size'][i] > 10 * df['bid_size'][i]: 
#             df['iceburg_bid'][i + 10] += df['ask_size'][i]
#     features.append(f'iceburg_bid')
    
    return df[features]
    

# Section 3: Model Selection CatBoost

In [ ]:
# TRAINING = True
# if TRAINING:
# #    df_train = pd.read_csv('/kaggle/input/optiver-trading-at-the-close/train.csv')
#    df_ = generate_features(train)

In [ ]:
TRAINING = True
if TRAINING:
   df_ = generate_features(train)

In [ ]:
model_path ='/kaggle/input/RKCP0219'
os.makedirs('models', exist_ok=True)

N_fold = 5

if TRAINING:
    X = df_.values
    Y = train['target'].values

    X = X[np.isfinite(Y)]
    Y = Y[np.isfinite(Y)]

    index = np.arange(len(X))

In [ ]:
models = []

def train(model_dict, modelname='lgb'):
    if TRAINING:
        model = model_dict[modelname]
        model.fit(X[index%N_fold!=i], Y[index%N_fold!=i], 
                    eval_set=[(X[index%N_fold==i], Y[index%N_fold==i])], 
                    verbose=10, 
                    early_stopping_rounds=100
                    )
        models.append(model)
        joblib.dump(model, './models/{modelname}_{i}.model')
    else:
        models.append(joblib.load(f'{model_path}/{modelname}_{i}.model'))
    return 

model_dict = {
    'cbt': cbt.CatBoostRegressor(objective='MAE', iterations=10),

}

for i in range(N_fold):
    train(model_dict, 'cbt')

In [ ]:
import optiver2023
env = optiver2023.make_env()
iter_test = env.iter_test()

In [ ]:
counter = 0
for (test, revealed_targets, sample_prediction) in iter_test:
    feat = generate_features(test)
    
    sample_prediction['target'] = np.mean([model.predict(feat) for model in models], 0)
    env.predict(sample_prediction)
    counter += 1